In [ ]:
from dotenv import load_dotenv

load_dotenv('../.env')

In [ ]:
import pandas as pd

df_testset = pd.read_csv('../ragas_openai_gpt4o-pt-corrigido.csv')
df_testset.rename(columns = {'reference': 'response'}, inplace = True)
df_testset.head(2)

In [ ]:
from langchain_aws import BedrockLLM

model_id = "meta.llama3-70b-instruct-v1:0"

custom_llm = BedrockLLM(
    region='us-east-1',
    model_id=model_id,  # ARN like 'arn:aws:bedrock:...' obtained via provisioning the custom model
    model_kwargs={"temperature": 0.0},
)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = """
Você é um servidor da UFRN responsável por instruir alunos sobre questões acadêmicas do regulamento dos cursos de graduação.
Você deve responder a resposta correta baseada na questão e contexto abaixo. Por favor, siga as instruções:

1. Pergunta: {pergunta}

2. Contexto: {contexto}

3. Instruções:
    - Analise cuidadosamente a questão e o contexto fornecido.
    - Formule uma resposta abrangente e precisa baseada apenas nas informações fornecidas no contexto.
    - Certifique-se de que sua resposta aborda diretamente a pergunta.
    - Inclua todas as informações relevantes do contexto, mas não adicione nenhum conhecimento externo.
    - Se o contexto não contiver informações suficientes para responder completamente à pergunta, declare isso claramente e forneça a melhor resposta parcial possível.
    - Use um tom formal e objetivo.

"""

prompt_template = PromptTemplate.from_template(prompt)

In [ ]:
from time import sleep

responses = []

for _, row in df_testset.iterrows():
    row.to_dict()
    prompt = prompt_template.format(pergunta=row['user_input'], contexto=row['reference_contexts'])
    try:
        response = custom_llm.invoke(prompt)
        print(response)
        responses.append(response)
    except:
        sleep(1)


In [ ]:
df_testset['reference'] = pd.Series(responses)
df_testset.head(2)

In [ ]:
df_testset.to_csv('../dataset_intermediario.csv')

In [ ]:
import pandas as pd

df_testset = pd.read_csv('../dataset_intermediario.csv')

In [ ]:
from datasets import Dataset

def pandas_to_ragas(df):
    '''
    Converts a Pandas DataFrame into a Ragas-compatible dataset
    
    Inputs:
        - df (Pandas DataFrame): The input DataFrame to be converted
        
    Returns:
        - ragas_testset (Hugging Face Dataset): A Hugging Face dataset compatible with the Ragas framework
    '''
    # Ensure all text columns are strings and handle NaN values
    text_columns = df.columns
    for col in text_columns:
        df[col] = df[col].fillna('').astype(str)
        
    # Convert 'contexts' to a list of lists
    df['reference_contexts'] = df['reference_contexts'].fillna('').astype(str).apply(lambda x: [x] if x else [])
    # df['retrieved_contexts'] = df['retrieved_contexts'].fillna('').astype(str).apply(lambda x: [x] if x else [])
    
    # Converting the DataFrame to a dictionary
    data_dict = df.to_dict('list')
    
    # Loading the dictionary as a Hugging Face dataset
    ragas_testset = Dataset.from_dict(data_dict)
    
    return ragas_testset

eval_dataset = pandas_to_ragas(df_testset)

In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas.metrics import NonLLMContextRecall
from ragas import evaluate

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [ ]:
metrics = [
    # LLMContextRecall(llm=evaluator_llm), 
    FactualCorrectness(llm=evaluator_llm), 
    # Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]
results = evaluate(dataset=eval_dataset, metrics=metrics)

In [ ]:
df_result = results.to_pandas()

In [ ]:
df_result[df_result['factual_correctness'] > 0.0]

In [ ]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics._string import NonLLMStringSimilarity

sample = SingleTurnSample(
    response="The Eiffel Tower is located in India.",
    reference="The Eiffel Tower is located in Paris."
)

scorer = NonLLMStringSimilarity()
await scorer.single_turn_ascore(sample)